In [ ]:
pip install tensorflow

In [ ]:
pip install torchtext


In [ ]:
pip install torch torchvision

In [ ]:
pip install nltk

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelBinarizer
import nltk
from nltk.tokenize import word_tokenize
# Read the text file
with open('./data/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()
    text = text.lower()

In [ ]:
print(text)

In [2]:
tokens = word_tokenize(text)
vocabulary = set(tokens)
total_words = len(vocabulary) + 1

word_to_idx = {word:idx for idx, word in enumerate(vocabulary)}

print(f"total_words: {total_words}")
print("Índice de palabras:", word_to_idx)

total_words: 8384
Índice de palabras: {"'hullo": 0, 'backward': 1, 'esq': 2, 'trusted': 3, 'adjective': 4, 'snigger': 5, 'quivering': 6, 'chose': 7, 'simon': 8, 'recovering': 9, 'fruitless': 10, 'prompt': 11, 'loftily': 12, 'ascended': 13, 'must': 14, 'cuts': 15, 'cloud-wreaths': 16, 'seat': 17, 'sweet': 18, 'faint': 19, 'liver': 20, 'themselves': 21, 'discover': 22, 'vulnerable': 23, 'commonplaces': 24, 'occasionally': 25, 'waterproof': 26, 'labour': 27, 'sight': 28, 'wondering': 29, 'setting': 30, 'upset': 31, 'guilt': 32, 'army': 33, 'orgies': 34, 'doing': 35, 'coincidences': 36, 'impossibility': 37, 'pittance': 38, 'fund': 39, 'glimpses': 40, 'pulled': 41, 'wickedness': 42, 'overtaken': 43, 'important': 44, 'grizzled': 45, 'stories': 46, 'satisfy': 47, 'daubing': 48, 'stevenson': 49, 'glanced': 50, 'hotel': 51, 'tear': 52, 'waxed': 53, 'killed': 54, "man's": 55, 'level': 56, 'gesticulating': 57, 'cast-off': 58, 'bed': 59, 'parley': 60, 'noose': 61, 'sucked': 62, 'attired': 63, 'car

In [3]:
input_sequences = []
for line in text.split('\n'):
    line_list = line.rstrip(",.;:").split(' ')
    token_list = []
    for char in line_list:
        if char in word_to_idx.keys():
            token_list.append(word_to_idx[char])

    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
# Imprimiendo las secuencias de n-gramas
print(f"Secuencias de n-gramas: {input_sequences[:5]}")

In [4]:
from torch.nn.utils.rnn import pad_sequence


In [5]:
# Determinamos la longitud máxima de las secuencias.
max_sequence_len = max(len(seq) for seq in input_sequences)

# Añadimos padding al principio de cada secuencia.
padded_sequences = []
for seq in input_sequences:
    num_padding = max_sequence_len - len(seq)
    padded_seq = [0] * num_padding + seq
    padded_sequences.append(padded_seq)

# Convertimos las secuencias en tensores.
tensor_sequences = torch.tensor(padded_sequences)



In [ ]:
print(f"max_sequence_len: {max_sequence_len}")
print(f"Secuencias de n-gramas: {tensor_sequences[:5]}")

In [6]:
X = tensor_sequences[:, :-1]
y = tensor_sequences[:, -1]

In [ ]:
print(X)
print(y)

In [7]:
import torch.nn.functional as F

In [8]:
# Convertir las etiquetas a one-hot encoding
num_classes = y.max().item() + 1  # Determinar el número de clases
y = F.one_hot(y, num_classes=num_classes)

In [ ]:
print("Secuencias con padding (X):")
print(X)
print("Etiquetas (y) en one-hot encoding:")
print(y)

In [9]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [10]:
dataset = TextDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

C:\Users\Alvaro\AppData\Local\Temp\ipykernel_27688\3547505943.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.long)


In [11]:
class LSTMModel(nn.Module):
    def __init__(self, total_words, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(total_words, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out , _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]
        x = self.fc(lstm_out)
        x = self.softmax(x)
        return x

In [22]:
model = LSTMModel(total_words, 100, 150, total_words).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [23]:
def train_model(model, dataloader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()  # Mover datos a la GPU
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, torch.max(y_batch, 1)[1])
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

In [24]:
import torch
torch.cuda.is_available()

a=torch.FloatTensor([1.0,2.0]).cuda()

a.device

torch.manual_seed(32)


next(model.parameters()).is_cuda

True

In [25]:
gpumodel = model.cuda()

In [26]:
train_model(model, dataloader, criterion, optimizer, epochs=100)

Epoch 1/100, Loss: 8.995824813842773
Epoch 2/100, Loss: 8.995824813842773
Epoch 3/100, Loss: 8.880439758300781
Epoch 4/100, Loss: 8.95736312866211
Epoch 5/100, Loss: 8.995824813842773
Epoch 6/100, Loss: 8.995824813842773
Epoch 7/100, Loss: 8.95736312866211
Epoch 8/100, Loss: 8.995824813842773
Epoch 9/100, Loss: 9.034285545349121
Epoch 10/100, Loss: 8.95736312866211
Epoch 11/100, Loss: 8.918901443481445
Epoch 12/100, Loss: 8.995824813842773
Epoch 13/100, Loss: 8.995824813842773
Epoch 14/100, Loss: 8.95736312866211
Epoch 15/100, Loss: 9.034285545349121
Epoch 16/100, Loss: 8.95736312866211
Epoch 17/100, Loss: 9.034285545349121
Epoch 18/100, Loss: 9.034285545349121
Epoch 19/100, Loss: 8.995824813842773
Epoch 20/100, Loss: 8.918901443481445
Epoch 21/100, Loss: 8.995824813842773
Epoch 22/100, Loss: 8.995824813842773
Epoch 23/100, Loss: 8.95736312866211
Epoch 24/100, Loss: 8.95736312866211
Epoch 25/100, Loss: 8.880439758300781
Epoch 26/100, Loss: 8.880439758300781
Epoch 27/100, Loss: 8.957363

In [ ]:
model.eval()
#Guardar modelo
torch.save(model.state_dict(), "model.pth")

#Guardar tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(word_to_idx, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [ ]:
# Texto inicial y número de palabras a predecir
seed_text = "I will leave if they"
next_words = 3

idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Tokenizamos el seed_text y lo convertimos a índices
tokens = word_tokenize(seed_text)
input_sequence = [word_to_idx[word] for word in tokens if word in word_to_idx]

# Generamos las siguientes palabras
model.eval()  # Configuramos el modelo en modo evaluación
for _ in range(next_words):
    # Convertimos la secuencia de entrada a tensor y añadimos una dimensión
    input_tensor = torch.tensor(input_sequence).unsqueeze(0)

    # Realizamos la predicción con el modelo
    with torch.no_grad():
        output = model(input_tensor)

    # Obtenemos el índice de la palabra predicha
    predicted_idx = torch.argmax(output, dim=1).item()

    # Añadimos la palabra predicha a la secuencia de entrada
    input_sequence.append(predicted_idx)

    # Añadimos la palabra predicha al texto semilla
    word = idx_to_word[predicted_idx]
    seed_text += " " + word

print(seed_text)

I will leave if they the the the
